# Dealing with errors after a run

In this example, we run the model on a list of three glaciers:
two of them will end with errors: one because it already failed at
preprocessing (i.e. prior to this run), and one during the run. We show how to analyze theses erros and solve (some) of them, as described in the OGGM documentation under [troubleshooting](https://docs.oggm.org/en/stable/faq.html?highlight=border#troubleshooting).

## Run with `cfg.PARAMS['continue_on_error'] = True`

In [4]:
# Locals
import oggm.cfg as cfg
from oggm import utils, workflow, tasks

# Libs
import os
import xarray as xr
import pandas as pd

# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')

# Here we override some of the default parameters
# How many grid points around the glacier?
# We make it small because we want the model to error because
# of flowing out of the domain
cfg.PARAMS['border'] = 80

# This is useful since we have three glaciers
cfg.PARAMS['use_multiprocessing'] = True

# This is the important bit!
# We tell OGGM to continue despite of errors
cfg.PARAMS['continue_on_error'] = True

# Local working directory (where OGGM will write its output)
WORKING_DIR = utils.gettempdir('OGGM_Errors')
utils.mkdir(WORKING_DIR, reset=True)
cfg.PATHS['working_dir'] = WORKING_DIR

rgi_ids = ['RGI60-11.00897', 'RGI60-11.01450', 'RGI60-11.03295']

# Go - get the pre-processed glacier directories
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4)

# We can step directly to the experiment!
# Random climate representative for the recent climate (1985-2015)
# with a negative bias added to the random temperature series
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=150, seed=0,
                             temperature_bias=-1)

2021-02-10 09:35:30: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2021-02-10 09:35:30: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2021-02-10 09:35:30: oggm.cfg: Multiprocessing: using all available processors (N=8)
2021-02-10 09:35:30: oggm.cfg: PARAMS['border'] changed from `20` to `80`.
2021-02-10 09:35:30: oggm.cfg: PARAMS['continue_on_error'] changed from `False` to `True`.
2021-02-10 09:35:30: oggm.workflow: init_glacier_directories from prepro level 4 on 3 glaciers.
2021-02-10 09:35:30: oggm.workflow: Execute entity task gdir_from_prepro on 3 glaciers
2021-02-10 09:35:30: oggm.workflow: Execute entity task run_random_climate on 3 glaciers
2021-02-10 09:35:36: oggm.core.flowline: RuntimeError occurred during task flowline_model_run on RGI60-11.00897: Glacier exceeds domain boundaries, at year: 98.08333333333333
2021-02-10 09:36:15: oggm.core.flowline: InvalidWorkflowError occurred during task run_random_climate on

## Error diagnostics

In [ ]:
# Write the compiled output
utils.compile_glacier_statistics(gdirs);  # saved as glacier_statistics.csv in the WORKING_DIR folder
utils.compile_run_output(gdirs);  # saved as run_output.nc in the WORKING_DIR folder

In [17]:
# Read it
with xr.open_dataset(os.path.join(WORKING_DIR, 'run_output.nc')) as ds:
    ds = ds.load()
df_stats = pd.read_csv(os.path.join(WORKING_DIR, 'glacier_statistics.csv'), index_col=0)

In [5]:
# all possible statistics about the glaciers
df_stats

,rgi_region,rgi_subregion,name,cenlon,cenlat,rgi_area_km2,rgi_year,glacier_type,terminus_type,is_tidewater,...,perc_invalid_flowline,inversion_glen_a,inversion_fs,error_task,error_msg,t_star,mu_star_glacierwide,mu_star_flowline_avg,mu_star_allsame,mb_bias
rgi_id,,,,,,,,,,,,,,,,,,,,,
RGI60-11.00897,11,11-01,Hintereisferner,10.75840,46.8003,8.036,2003,Glacier,Land-terminating,False,...,0.172727,1.322172e-23,0.0,flowline_model_run,RuntimeError: Glacier exceeds domain boundarie...,1974.0,221.276187,221.276187,True,4.685993
RGI60-11.01450,11,11-01,,8.01919,46.5028,82.206,2003,Glacier,Land-terminating,False,...,0.000000,1.322172e-23,0.0,None,None,1929.0,294.283491,294.283491,True,7.271712
RGI60-11.03295,11,11-01,Fr4N01146B09 De Rosolin,6.84300,45.4170,0.600,2003,Glacier,Land-terminating,False,...,NaN,NaN,NaN,glacier_masks,InvalidGeometryError: This glacier geometry is...,NaN,NaN,NaN,NaN,NaN


- in the column *error_task*, we can see whether an error occurred, and if yes during which task
- *error_msg* describes the actual error message 

In [7]:
df_stats[['error_task', 'error_msg']]

,error_task,error_msg
rgi_id,,
RGI60-11.00897,flowline_model_run,RuntimeError: Glacier exceeds domain boundarie...
RGI60-11.01450,None,None
RGI60-11.03295,glacier_masks,InvalidGeometryError: This glacier geometry is...


We can also check which glacier failed at which task by using [compile_task_log]('https://docs.oggm.org/en/stable/generated/oggm.utils.compile_task_log.html#oggm.utils.compile_task_log').

In [13]:
# also saved as task_log.csv in the WORKING_DIR folder - "append=False" replaces the existing one
utils.compile_task_log(gdirs, task_names=['glacier_masks', 'compute_centerlines', 'flowline_model_run'], append=False)

,glacier_masks,compute_centerlines,flowline_model_run
rgi_id,,,
RGI60-11.00897,SUCCESS,SUCCESS,RuntimeError: Glacier exceeds domain boundarie...
RGI60-11.01450,SUCCESS,SUCCESS,SUCCESS
RGI60-11.03295,InvalidGeometryError: This glacier geometry is...,FileNotFoundError: [Errno 2] No such file or d...,


## Error solving

### RuntimeError: `Glacier exceeds domain boundaries, at year: 98.08333333333333`

To remove this error just increase the domain boundary **before** running `init_glacier_directories` ! Attention, this means that more data has to be downloaded and the run takes more time. The available options for `cfg.PARAMS['border']` are **10, 40, 80 or 160** at the moment; the unit is number of grid points outside the glacier boundaries. More about that in the OGGM documentation under [preprocessed files](https://docs.oggm.org/en/stable/input-data.html#pre-processed-directories).

In [14]:
# reset to recompute statistics
utils.mkdir(WORKING_DIR, reset=True)

# increase the amount of gridpoints outside the glacier
cfg.PARAMS['border'] = 160
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=4)
workflow.execute_entity_task(tasks.run_random_climate, gdirs,
                             nyears=150, seed=0,
                             temperature_bias=-1);

# recompute the output
# we can also get the run output directly from the methods
df_stats = utils.compile_glacier_statistics(gdirs)
ds = utils.compile_run_output(gdirs)

2021-02-10 10:00:37: oggm.cfg: PARAMS['border'] changed from `80` to `160`.
2021-02-10 10:00:37: oggm.workflow: init_glacier_directories from prepro level 4 on 3 glaciers.
2021-02-10 10:00:37: oggm.workflow: Execute entity task gdir_from_prepro on 3 glaciers
100% of  24.6 MiB |######################| Elapsed Time: 0:00:10 Time:  0:00:10
100% of  24.4 MiB |######################| Elapsed Time: 0:00:05 Time:  0:00:05
100% of  22.7 MiB |######################| Elapsed Time: 0:00:04 Time:  0:00:04
2021-02-10 10:00:59: oggm.workflow: Execute entity task run_random_climate on 3 glaciers
2021-02-10 10:01:52: oggm.core.flowline: InvalidWorkflowError occurred during task run_random_climate on RGI60-11.03295: Need a valid `model_flowlines` file. If you explicitly want to use `inversion_flowlines`, set use_inversion_flowlines=True.
2021-02-10 10:01:52: oggm.workflow: Execute entity task glacier_statistics on 3 glaciers
2021-02-10 10:01:52: oggm.utils: Applying compile_run_output on 3 gdirs.


In [18]:
# check again
df_stats[['error_task', 'error_msg']]

,error_task,error_msg
rgi_id,,
RGI60-11.00897,NaN,NaN
RGI60-11.01450,NaN,NaN
RGI60-11.03295,glacier_masks,InvalidGeometryError: This glacier geometry is...


Now `RGI60-11.00897` runs without errors!

### Error: `Need a valid model_flowlines file.`

This error message in the log is misleading: it does not really describe the source of the error, which happened earlier in the processing chain. Therefore we can look instead into the glacier_statistics via [compile_glacier_statistics](https://docs.oggm.org/en/stable/generated/oggm.utils.compile_glacier_statistics.html) or into the log output via [compile_task_log](https://docs.oggm.org/en/stable/generated/oggm.utils.compile_task_log.html#oggm.utils.compile_task_log):

In [19]:
print('error_task: {}, error_msg: {}'.format(df_stats.loc['RGI60-11.03295']['error_task'],
                                             df_stats.loc['RGI60-11.03295']['error_msg']))

error_task: glacier_masks, error_msg: InvalidGeometryError: This glacier geometry is not valid for OGGM.


Now we have a better understanding of the error: 
- OGGM can not work with this geometry of this glacier and could therefore not make a gridded mask of the glacier outlines. 
- there is no way to prevent this except you find a better way to pre-process the geometry of this glacier
- these glaciers have to be ignored! Less than 0.5% of glacier area globally have errors during the geometry processing or failures in computing certain topographical properties by e.g. invalid DEM, see [Sect. 4.2 Invalid Glaciers of the OGGM paper (Maussion et al., 2019)](https://gmd.copernicus.org/articles/12/909/2019/#section4) and [this tutorial](preprocessing_errors.ipynb) for more up-to-date numbers

## Ignoring those glaciers with errors that we can't solve

In the run_output, you can for example just use `*.dropna` to remove these. For other applications (e.g. quantitative mass change evaluation), more will be needed (not available yet in the OGGM codebase):

In [20]:
ds.dropna(dim='rgi_id')  # here we can e.g. find the volume evolution

<xarray.Dataset>
Dimensions:         (rgi_id: 2, time: 151)
Coordinates:
  * time            (time) float64 0.0 1.0 2.0 3.0 ... 147.0 148.0 149.0 150.0
  * rgi_id          (rgi_id) object 'RGI60-11.00897' 'RGI60-11.01450'
    hydro_year      (time) int64 0 1 2 3 4 5 6 7 ... 144 145 146 147 148 149 150
    hydro_month     (time) int64 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1
    calendar_year   (time) int64 -1 0 1 2 3 4 5 ... 143 144 145 146 147 148 149
    calendar_month  (time) int64 10 10 10 10 10 10 10 ... 10 10 10 10 10 10 10
Data variables:
    volume          (time, rgi_id) float32 6.424e+08 1.285e+10 ... 1.676e+10
    volume_bsl      (time, rgi_id) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    volume_bwl      (time, rgi_id) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    area            (time, rgi_id) float32 8.036e+06 8.221e+07 ... 9.301e+07
    length          (time, rgi_id) float32 6.9e+03 2.329e+04 ... 3.178e+04
    calving         (time, rgi_id) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    calving_rate    (time, rgi_id) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    water_level     (rgi_id) float32 0.0 0.0
    glen_a          (rgi_id) float32 1.324e-23 1.324e-23
    fs              (rgi_id) float32 0.0 0.0
Attributes:
    description:    OGGM model output
    oggm_version:   1.3.2.dev142+g5fe880a
    calendar:       365-day no leap
    creation_date:  2021-02-10 09:01:52

## What's next?

- read about [preprocessing errors](preprocessing_errors.ipynb)
- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](welcome.ipynb)